In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
# from autosklearn.classification import AutoSklearnClassifier
import scipy.stats
import pandas as pd
import numpy as np
import util.Data
# import util.Plotly as plots
# import util.colors.Colors as colors

In [6]:
#dataset
csv_filename = r'../repr/data/final_coordinates-conversion-v2.csv'
df_data = util.Data.import_data(csv_filename)

In [7]:
df_data.shape

(656, 26)

In [8]:
df_data.dtypes

importance        object
author            object
title             object
table_name        object
keywords          object
Left/Right        object
Name              object
Broadman Area     object
t                float64
X(R)               int64
Y(A)               int64
Z(S)               int64
z-score          float64
NV               float64
p value          float64
cluster size     float64
index_col          int64
MNIX               int64
MNIY               int64
MNIZ               int64
BA                object
TALX               int64
TALY               int64
TALZ               int64
BA_name           object
BA_no            float64
dtype: object

## Dataframe valid

In [9]:
mask_outside_all = df_data['BA_name'].str.contains('Outside defined BAs')
df_data_valid = df_data[~mask_outside_all]

In [10]:
df_data_valid.shape

(483, 26)

In [11]:
set_keywords_selection = set(
    ["reward magnitude", "anticipation phase", "outcome phase", "decision phase", "working memory", "viewing",
     "monetary reward", "high gain vs high loss", "taste reward", "social reward", "monetary and taste reward", "food ",
     "high calorie", "low calorie", "positive reward", "negative reward", "context-dependent reward", "risk choice",
     "safe choice"])

In [12]:
df = df_data_valid
counter = {}
for val in set_keywords_selection:
    counter[val] = df['keywords'][df['keywords'].str.contains(val)].count()
counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1])}
counter

{'working memory': 0,
 'taste reward': 0,
 'monetary and taste reward': 0,
 'safe choice': 0,
 'high calorie': 5,
 'risk choice': 14,
 'high gain vs high loss': 14,
 'decision phase': 18,
 'low calorie': 18,
 'food ': 18,
 'negative reward': 23,
 'viewing': 24,
 'context-dependent reward': 27,
 'anticipation phase': 76,
 'social reward': 84,
 'positive reward': 97,
 'reward magnitude': 146,
 'monetary reward': 277,
 'outcome phase': 378}

In [33]:
# refine keywords
min_val = 20
set_keywords = [key for key in set_keywords_selection if counter[key] >= min_val]
set_keywords

['outcome phase',
 'reward magnitude',
 'monetary reward',
 'positive reward',
 'negative reward',
 'social reward',
 'anticipation phase',
 'viewing',
 'context-dependent reward']

In [14]:
# significance
def significance(row, df=10):
    if not (pd.isna(row['p value'])):
        return abs(row['p value'])
    elif not (pd.isna(row['z-score'])):
        return scipy.stats.norm.sf(row['z-score'])
    elif not (pd.isna(row['t'])):
        return scipy.stats.t.sf(row['t'], df)
    return 0

In [15]:
df_data_valid['significance'] = df_data_valid.apply(significance, axis=1)

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
# Create y
y = []
x = []
for _, row in df_data_valid.iterrows():
    x.append(np.array([row['MNIX'], row['MNIY'], row['MNIZ']]))
    y.append(
        # [round(1 * (1 - row['significance']), 4) if key in row['keywords'] else 0 for key in set_keywords_selection])
        np.array([1 if key in row['keywords'] else 0 for key in set_keywords]))
y = np.array(y)
x = np.array(x)

In [17]:
# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=23)

In [18]:
x

array([[-28, -24, -14],
       [-21,  21, -25],
       [  4, -56,  35],
       ...,
       [-59,  -3, -18],
       [ 38,  39, -17],
       [ 56,  -6, -17]])

In [15]:
# Auto-Sklearn Initialization

# time_left_for_this_task : Time limit in seconds to find the optimal configuration
# per_run_time_limit : Time limit in seconds for each model
# ensemble_size: Number of models added to the Ensemble model
# initial_configurations_via_metalearning: "k" configurations to start the Bayesian Optimization
model = AutoSklearnClassifier(time_left_for_this_task=300,
                              per_run_time_limit=30,
                              ensemble_size=1,
                              initial_configurations_via_metalearning=0)
# Init training
model.fit(x_train, y_train)

[WARNING] [2022-05-13 10:25:04,128:Client-AutoML(1):2f881543-d296-11ec-8c79-fdce89fc766c] Unknown dtype for X: int64, assuming it takes 8 bit/number


AutoSklearnClassifier(ensemble_size=1,
                      initial_configurations_via_metalearning=0,
                      per_run_time_limit=30, time_left_for_this_task=300)

In [16]:
print(model.sprint_statistics())

auto-sklearn results:
  Dataset name: 2f881543-d296-11ec-8c79-fdce89fc766c
  Metric: f1_macro
  Best validation score: 0.412912
  Number of target algorithm runs: 161
  Number of successful target algorithm runs: 150
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



In [17]:
print(model.leaderboard())

          rank  ensemble_weight          type      cost  duration
model_id                                                         
160          1              1.0  bernoulli_nb  0.587088  0.433002


In [18]:
print(model.show_models())

{160: {'model_id': 160, 'rank': 1, 'cost': 0.587088389492443, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f4b1785cd00>, 'balancing': Balancing(random_state=1, strategy='weighting'), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f4b1785c610>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f4b17e67340>, 'sklearn_classifier': OneVsRestClassifier(estimator=BernoulliNB(alpha=0.021354685179664025), n_jobs=1)}}


In [19]:
import sklearn.metrics

In [27]:
test_predictions = model.predict(x_test)
print("mse:", sklearn.metrics.mean_squared_error(y_test, test_predictions))
print("acc:", sklearn.metrics.accuracy_score(y_test, test_predictions))

mse: 0.2952815829528158
acc: 0.0273972602739726


In [21]:
test_predictions

array([[1, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 1,

In [22]:
y_test

array([[0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0,

## Multioutput regression

In [34]:
# Create y
y = []
x = []
for _, row in df_data_valid.iterrows():
    x.append(np.array([row['MNIX'], row['MNIY'], row['MNIZ']]))
    y.append(
        [round(1 * (1 - row['significance']), 4) if key in row['keywords'] else 0 for key in set_keywords])
        # np.array([1 if key in row['keywords'] else 0 for key in set_keywords]))
y = np.array(y)
x = np.array(x)

In [50]:
# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=23)

In [55]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
model = MultiOutputRegressor(Ridge(random_state=0)).fit(x_train, y_train)
y_predict = model.predict(x_test)

In [56]:
y_predict[0]

array([0.74033256, 0.37447395, 0.46454721, 0.2175136 , 0.06521734,
       0.22801904, 0.16645307, 0.07614977, 0.02803063])

In [57]:
y_test[0]

array([0.9998, 0.    , 0.9998, 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    ])

In [58]:
print("mse:", sklearn.metrics.mean_squared_error(y_test, y_predict))

mse: 0.11903579008200985


In [63]:
x_test[0]

array([-18,  44, -22])

In [59]:
x_tst2=[
    [-18,44,-22],
    [-17,44,-22],
    [-19,44,-22],
    [-18,43,-22],
    [-18,45,-22],
    [-18,44,-21],
    [-18,44,-23],
    [-17,43,-21],
    [-18,45,-23],
]

In [60]:
y_pred2 = model.predict(x_tst2)

In [61]:
y_pred2


array([[0.74033256, 0.37447395, 0.46454721, 0.2175136 , 0.06521734,
        0.22801904, 0.16645307, 0.07614977, 0.02803063],
       [0.73995163, 0.37372887, 0.46423065, 0.21700209, 0.06572126,
        0.22820099, 0.1655684 , 0.07578724, 0.02808848],
       [0.74071349, 0.37521904, 0.46486378, 0.21802511, 0.06471341,
        0.22783709, 0.16733775, 0.0765123 , 0.02797277],
       [0.74013144, 0.37606738, 0.46265601, 0.21617757, 0.06466281,
        0.22843539, 0.16735325, 0.07634384, 0.02735034],
       [0.74053368, 0.37288052, 0.46643842, 0.21884963, 0.06577187,
        0.2276027 , 0.16555289, 0.0759557 , 0.02871092],
       [0.74110554, 0.37119288, 0.47015143, 0.21917937, 0.06524058,
        0.22546067, 0.1656135 , 0.07515535, 0.02982978],
       [0.73955958, 0.37775503, 0.458943  , 0.21584783, 0.06519409,
        0.23057742, 0.16729264, 0.07714419, 0.02623147],
       [0.74052348, 0.37204122, 0.46794365, 0.21733183, 0.06518998,
        0.22605897, 0.165629  , 0.07498689, 0.02920735],
